<a href="https://colab.research.google.com/github/ParzvaI/decision_tree/blob/main/_notebooks/2020-06-03-01-Decision-tree-for-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
wbc = pd.read_csv('./data.csv')
wbc.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
X = wbc[['radius_mean', 'concave points_mean']]
y = wbc['diagnosis']
y = y.map({'M':1, 'B':0})

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [5]:
from sklearn.tree import DecisionTreeClassifier

# Instantiate a DecisionTreeClassifier 'dt' with a maximum depth of 6
dt = DecisionTreeClassifier(max_depth=6, random_state=1)

# Fit dt to the training set
dt.fit(X_train, y_train)

# Predict test set labels
y_pred = dt.predict(X_test)
print(y_pred[0:5])

[1 0 0 1 0]


In [6]:
from sklearn.metrics import accuracy_score

# Predict test set labels
y_pred = dt.predict(X_test)

# Compute test set accuracy
acc = accuracy_score(y_test, y_pred)
print("Test set accuracy: {:.2f}".format(acc))

Test set accuracy: 0.89


In [13]:
from sklearn.tree import DecisionTreeClassifier

# Instantiate dt_entropy, set 'entropy' as the information criterion
dt_entropy = DecisionTreeClassifier(max_depth=3, criterion='entropy', random_state=1)

# Fit dt_entropy to the training set
dt_entropy.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=1)

In [14]:
dt_gini = DecisionTreeClassifier(max_depth=8, criterion='gini', random_state=1)
dt_gini.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8, random_state=1)

In [15]:
from sklearn.metrics import accuracy_score

# Use dt_entropy to predict test set labels
y_pred = dt_entropy.predict(X_test)
y_pred_gini = dt_gini.predict(X_test)

# Evaluate accuracy_entropy
accuracy_entropy = accuracy_score(y_test, y_pred)
accuracy_gini = accuracy_score(y_test, y_pred_gini)

# Print accuracy_entropy
print("Accuracy achieved by using entropy: ", accuracy_entropy)

# Print accuracy_gini
print("Accuracy achieved by using gini: ", accuracy_gini)

Accuracy achieved by using entropy:  0.9122807017543859
Accuracy achieved by using gini:  0.8859649122807017


In [51]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Generate the if-else rules
tree_rules = export_text(dt_gini, feature_names=list(X.columns))
print(tree_rules)  # Print the rules

|--- concave points_mean <= 0.05
|   |--- radius_mean <= 15.03
|   |   |--- class: 0
|   |--- radius_mean >  15.03
|   |   |--- class: 1
|--- concave points_mean >  0.05
|   |--- radius_mean <= 15.16
|   |   |--- class: 1
|   |--- radius_mean >  15.16
|   |   |--- class: 1



In [53]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    def recurse(node, depth):
        indent = "    " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print(f"{indent}if features['{name}'] <= {threshold:.2f}:")
            recurse(tree_.children_left[node], depth + 1)
            print(f"{indent}else:  # if features['{name}'] > {threshold:.2f}")
            recurse(tree_.children_right[node], depth + 1)
        else:
            print(f"{indent}return {tree_.value[node].argmax()}")

    print("def decision_tree_predict(features):")
    recurse(0, 1)

# Generate Python function from Decision Tree
tree_to_code(dt_gini, list(X.columns))

def decision_tree_predict(features):
    if features['concave points_mean'] <= 0.05:
        if features['radius_mean'] <= 15.03:
            return 0
        else:  # if features['radius_mean'] > 15.03
            return 1
    else:  # if features['concave points_mean'] > 0.05
        if features['radius_mean'] <= 15.16:
            return 1
        else:  # if features['radius_mean'] > 15.16
            return 1


In [54]:
def decision_tree_predict(features):
    if features['concave points_mean'] <= 0.05:
        if features['radius_mean'] <= 15.03:
            return 0
        else:  # if features['radius_mean'] > 15.03
            return 1
    else:  # if features['concave points_mean'] > 0.05
        if features['radius_mean'] <= 15.16:
            return 1
        else:  # if features['radius_mean'] > 15.16
            return 1

In [55]:
X_test[:5]

,radius_mean,concave points_mean
421,14.69,0.06300
47,13.17,0.07340
292,12.95,0.03370
186,18.31,0.05814
414,15.13,0.02739


In [56]:
y_pred[:5]

array([1, 1, 0, 1, 1])

In [57]:
print(decision_tree_predict({"radius_mean":14.69,"concave points_mean":0.06300}))

1


In [58]:
print(decision_tree_predict({"radius_mean":13.17,"concave points_mean":0.07340}))

1


In [59]:
print(decision_tree_predict({"radius_mean":12.95,"concave points_mean":0.03370}))

0


In [60]:
print(decision_tree_predict({"radius_mean":18.31,"concave points_mean":0.05814}))

1


In [61]:
print(decision_tree_predict({"radius_mean":15.13,"concave points_mean":0.02739}))

1


In [66]:
import time

start = time.time()
dt_gini.predict([[14.69,0.06300]])
end = time.time()
elapsed = end - start
print(f'Time taken: {elapsed:.6f} seconds')

Time taken: 0.001758 seconds


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [67]:
import time

start = time.time()
decision_tree_predict({"radius_mean":14.69,"concave points_mean":0.06300})
end = time.time()
elapsed = end - start
print(f'Time taken: {elapsed:.6f} seconds')

Time taken: 0.000161 seconds


In [73]:
dt_gini = DecisionTreeClassifier(max_depth=2,
                                 min_samples_split=5,
                                 min_samples_leaf=5,
                                 ccp_alpha=0.01,
                                 criterion='gini',
                                 random_state=1)
dt_gini.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.01, max_depth=2, min_samples_leaf=5,
                       min_samples_split=5, random_state=1)

In [74]:

# Instantiate dt_entropy, set 'entropy' as the information criterion
dt_entropy = DecisionTreeClassifier(max_depth=2,
                                 min_samples_split=5,
                                 min_samples_leaf=5,
                                 ccp_alpha=0.01,
                                 criterion='entropy',
                                 random_state=1)
# Fit dt_entropy to the training set
dt_entropy.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.01, criterion='entropy', max_depth=2,
                       min_samples_leaf=5, min_samples_split=5, random_state=1)

In [75]:
from sklearn.metrics import accuracy_score

# Use dt_entropy to predict test set labels
y_pred = dt_entropy.predict(X_test)
y_pred_gini = dt_gini.predict(X_test)

# Evaluate accuracy_entropy
accuracy_entropy = accuracy_score(y_test, y_pred)
accuracy_gini = accuracy_score(y_test, y_pred_gini)

# Print accuracy_entropy
print("Accuracy achieved by using entropy: ", accuracy_entropy)

# Print accuracy_gini
print("Accuracy achieved by using gini: ", accuracy_gini)

Accuracy achieved by using entropy:  0.9210526315789473
Accuracy achieved by using gini:  0.9210526315789473


In [72]:
tree_rules = export_text(dt_gini, feature_names=list(X.columns))
print(tree_rules)  # Print the rules

|--- concave points_mean <= 0.05
|   |--- radius_mean <= 15.03
|   |   |--- class: 0
|   |--- radius_mean >  15.03
|   |   |--- class: 1
|--- concave points_mean >  0.05
|   |--- radius_mean <= 15.16
|   |   |--- class: 1
|   |--- radius_mean >  15.16
|   |   |--- class: 1



In [76]:
tree_to_code(dt_gini, list(X.columns))

def decision_tree_predict(features):
    if features['concave points_mean'] <= 0.05:
        if features['radius_mean'] <= 15.03:
            return 0
        else:  # if features['radius_mean'] > 15.03
            return 1
    else:  # if features['concave points_mean'] > 0.05
        if features['radius_mean'] <= 15.16:
            return 1
        else:  # if features['radius_mean'] > 15.16
            return 1


In [77]:
def decision_tree_predict(features):
    if features['concave points_mean'] <= 0.05:
        if features['radius_mean'] <= 15.03:
            return 0
        else:  # if features['radius_mean'] > 15.03
            return 1
    else:  # if features['concave points_mean'] > 0.05
        if features['radius_mean'] <= 15.16:
            return 1
        else:  # if features['radius_mean'] > 15.16
            return 1

In [79]:
import time

start = time.time()
decision_tree_predict({"radius_mean":14.69,"concave points_mean":0.06300})
end = time.time()
elapsed = end - start
print(f'Time taken: {elapsed:.6f} seconds')

Time taken: 0.000119 seconds
